In [1]:
import math
import numpy as np
import scipy as sc
import scipy.optimize
import os
from pdf2image import convert_from_path
import cv2

Function 'find QRs' from pyexams.scan

In [2]:
def find_qrs(file, dpi):
    # TODO: improve qr detection
    pages = convert_from_path(file, dpi=dpi, thread_count=1, fmt='png')
    img_file = 'img_temp.png'
    qrs = []
    for page in pages:
        page.save(img_file)
        img = cv2.imread(img_file)
        qr_detect = cv2.QRCodeDetector()
        # process the image
        _, img_th = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
        # keep only black pixels
        hsv_img = cv2.cvtColor(img_th, cv2.COLOR_BGR2HSV)
        lower_values = np.array([0, 0, 0])
        upper_values = np.array([180, 255, 30])
        black_mask = cv2.inRange(hsv_img, lower_values, upper_values)
        # blur, sharpen and recognize the qr
        blur = cv2.GaussianBlur(black_mask, (3, 3), 0)
        sharpen = cv2.filter2D(blur, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))
        value, coordinates, qr = qr_detect.detectAndDecode(~sharpen)
        # if it doesn't find the qr, loop blur and sharpen until it does, up to 5 times
        count = 0
        while qr is None:
            if count == 5:
                # TODO: deal with QR not found error
                # TODO: raise exception
                error = 'Error: QR not found'
                break
            blur = cv2.GaussianBlur(sharpen, (3, 3), 0)
            sharpen = cv2.filter2D(blur, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))
            value, coordinates, qr = qr_detect.detectAndDecode(~sharpen)
            count = count + 1
        exam, variant, page = '', '', ''
        if value:
            exam, variant, page = value.split(sep=',')
        qrs.append({'exam': exam, 'variant': variant, 'page': page, 'coordinates': coordinates})
    if os.path.exists(img_file):
        os.remove(img_file)
    return qrs

In [3]:
dpi = 300

pang's functions

In [4]:
def rot(alpha):
    '''clockwise rotation

    :param: alpha in 360 degrees
    '''
    alpha_rad = alpha*np.pi/180
    return np.array([
        [np.cos(alpha_rad), np.sin(alpha_rad)],
        [-np.sin(alpha_rad), np.cos(alpha_rad)]
    ])

In [54]:
def trans(C1, C2):
    def sq(xs):
        x0, y0, scale, angle = xs
        C1_trans = np.array([[x0,y0]]) + scale*C1@rot(angle)
        return ((C1_trans - C2)**2).sum()
    x0_guess, y0_guess = C2[0,:] - C1[0,:]
    sol = sc.optimize.minimize(sq, (x0_guess, y0_guess, 1, 0), tol=1e-9, method='Powell')
    # if not sol['success']:
      #   raise RuntimeError('Numerical error: could not find transformation matrix')
    return tuple(sol['x'])

files with the exams

In [6]:
ex_file1 = '../multiple_choice_example/scanned/scanned_11111.pdf'
ex_file2 = '../multiple_choice_example/scanned/scanned_22222.pdf'

# filled ones should have the same coords
ex_file3 = '../multiple_choice_example/filled/filled_33333.pdf'
ex_file4 = '../multiple_choice_example/filled/filled_44444.pdf'

# filled 3 rotated by 90º and 180º
ex_file2_rot90 = '../multiple_choice_example/scanned/scanned_22222_rot90.pdf'
ex_file2_rot180 = '../multiple_choice_example/scanned/scanned_22222_rot180.pdf'
ex_file3_rot90 = '../multiple_choice_example/filled/filled_33333_rot90.pdf'
ex_file3_rot180 = '../multiple_choice_example/filled/filled_33333_rot180.pdf'

get the qrs from filled3, should be the default coordinates

In [ ]:
qrs3 = find_qrs(ex_file3, dpi)

In [7]:
qr_cords3 = np.zeros((4,2))
# qr_cords3, qr_cords3_90, qr_cords3_180 = np.zeros((12,2)), np.zeros((12,2)), np.zeros((12,2))
for page in range(1):
    for point in range(4):
        qr_cords3[page*4+point] = (qrs3[page]['coordinates'][0][point])
qr_cords3

array([[1950.00012207, 3174.        ],
       [2107.        , 3174.        ],
       [2107.        , 3331.        ],
       [1950.00012207, 3331.        ]])

get the qrs from the rotated scanned2 pdfs

In [ ]:
# qrs2 = find_qrs(ex_file2, dpi)
qrs2_rot90 = find_qrs(ex_file2_rot90, dpi)
# qrs2_rot180 = find_qrs(ex_file2_rot180, dpi)

In [9]:
# qr_cords2_p1 = np.zeros((4,2))
# qr_cords2_p2 = np.zeros((4,2))
# qr_cords2_p3 = np.zeros((4,2))
qr_cords2_90_p1 = np.zeros((4,2))
qr_cords2_90_p2 = np.zeros((4,2))
qr_cords2_90_p3 = np.zeros((4,2))
# qr_cords2_180_p1 = np.zeros((4,2))
# qr_cords2_180_p2 = np.zeros((4,2))
# qr_cords2_180_p3 = np.zeros((4,2))
for page in range(3):
    for point in range(4):
        if page == 0:
            qr_cords2_90_p1[point] = (qrs2_rot90[page]['coordinates'][0][point])
        elif page == 1:
            qr_cords2_90_p2[point] = (qrs2_rot90[page]['coordinates'][0][point])
        else:
            qr_cords2_90_p3[point] = (qrs2_rot90[page]['coordinates'][0][point])
qr_cords2_90_p1

array([[3101.        ,  570.        ],
       [3098.00390625,  419.27383423],
       [3252.        ,  414.        ],
       [3252.        ,  570.92858887]])

In [55]:
trans(qr_cords2_90_p1, qr_cords3)

(2573.194992703396, 4.6128973057080955, 1.0246963698062097, 90.68653678679813)

In [56]:
trans(qr_cords3, qr_cords2_90_p1)

(2381.4436705145554,
 2786.4511493428336,
 0.6329472983124579,
 -128.86905208897727)

In [57]:
trans(qr_cords2_90_p2, qr_cords3)

(2524.315722002431, -38.34697309151104, 1.0511263923004655, 90.13630513396531)

In [58]:
trans(qr_cords3, qr_cords2_90_p2)

(2359.2610313199343,
 2544.301171960416,
 0.5788193258940987,
 -127.58906165592911)

In [59]:
trans(qr_cords2_90_p3, qr_cords3)

(2551.8734027807623, -26.666768871652682, 1.0342671724712933, 90.2850213725594)

In [60]:
trans(qr_cords3, qr_cords2_90_p3)

(2384.911691975397, 2673.3933096615574, 0.6053845270428454, -128.1279298943979)

doesn't always work

# Defined a new function:

- get_rotation: compares two vectors and returns the angle they form and the difference in scale between them
- get_trans:
- 1: calls get_rotation with each two vectors made by joining each point with the next for both point arrays
- 2: finds the average of all angle/scale found
- 3:

In [40]:
def get_rotation(vector, vector_trans):
    prod = vector[0] * vector_trans[0] + vector[1] * vector_trans[1]
    d1 = math.sqrt(math.pow(vector[0], 2) + math.pow(vector[1], 2))
    d2 = math.sqrt(math.pow(vector_trans[0], 2) + math.pow(vector_trans[1], 2))
    cos = prod / (d1 * d2)
    cross_prod = vector[0] * vector_trans[1] - vector[1] * vector_trans[0]
    sign = 1 if cross_prod >= 0 else -1
    scale = d2 / d1
    return sign * math.acos(cos) * 180 / np.pi, scale

In [61]:
def get_trans(points, points_trans):
    num_points = points.__len__()
    if not num_points == points_trans.__len__():
        return ()
    tr_cords = np.zeros((4,2))
    alpha_final, scale_final, dist_final = 0, 0, 0
    # find rotation alpha and scale
    for i in range(num_points):
        if i == num_points - 1:
            vector = [points[0,0] - points[i,0], points[0,1] - points[i,1]]
            vector_trans = [points_trans[0,0] - points_trans[i,0], points_trans[0,1] - points_trans[i,1]]
        else:
            vector = [points[i+1,0] - points[i,0], points[i+1,1] - points[i,1]]
            vector_trans = [points_trans[i+1,0] - points_trans[i,0], points_trans[i+1,1] - points_trans[i,1]]
        alpha, scale = get_rotation(vector, vector_trans)
        # print(alpha, scale)
        alpha_final = alpha_final + alpha / num_points
        scale_final = scale_final + scale / num_points
    # find translation
    for i in range(num_points):
        tr_cords[i] = scale_final * qr_cords3[i] @ rot(alpha_final)
        dist = qr_cords2_90_p1[i] - tr_cords[i]
        # print(dist)
        dist_final = dist_final + dist / num_points
    print(dist_final, alpha_final, scale_final)
    return tr_cords + dist_final

In [49]:
get_trans(qr_cords3, qr_cords2_90_p1)

[  26.06753721 2510.77703129] -90.68695862866194 0.9757547714679142


array([[3100.07808099,  571.06013963],
       [3098.24138387,  417.87777046],
       [3251.42387214,  416.04107192],
       [3253.26056925,  569.22344108]])

In [43]:
qr_cords2_90_p1

array([[3101.        ,  570.        ],
       [3098.00390625,  419.27383423],
       [3252.        ,  414.        ],
       [3252.        ,  570.92858887]])

In [62]:
get_trans(qr_cords3, qr_cords2_90_p2)

[  86.2575113  2430.41796271] -90.13372761525127 0.9512699983867006


array([[3101.25077457,  568.39932881],
       [3100.9021956 ,  419.05046198],
       [3250.25117855,  418.70188273],
       [3250.59975753,  568.05074957]])

In [63]:
qr_cords2_90_p2

array([[3059.        ,  539.        ],
       [3056.60205078,  389.        ],
       [3205.97680664,  389.        ],
       [3206.        ,  540.        ]])

In [64]:
get_trans(qr_cords3, qr_cords2_90_p3)

[  41.31270448 2470.07575117] -90.28413336980311 0.9667029678149448


array([[3100.24204921,  569.81211945],
       [3099.48940418,  418.04173771],
       [3251.25990392,  417.2890921 ],
       [3252.01254894,  569.05947383]])

In [65]:
qr_cords2_90_p3

array([[3098.        ,  567.        ],
       [3096.        ,  416.01947021],
       [3251.        ,  413.        ],
       [3247.        ,  565.        ]])